# Simple component separation

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import healpy as hp
import pysm

from fgbuster.observation_helpers import get_instrument, get_sky
from fgbuster.component_model import CMB, Dust, Synchrotron
from fgbuster.separation_recipes import basic_comp_sep
from fgbuster.visualization import corner_norm

## Simulate your frequency maps
You have some frequecy maps to clean, they can be either data or simulations. For example, you can use `pysm` to simulate microwave skys with different complexities. ForeGroundBuster adds a couple of functions that make the process even easier.

In [ ]:
nside = 32
sky = pysm.Sky(get_sky(nside, 'c1d0s0'))
instrument = pysm.Instrument(get_instrument('litebird', nside))
freq_maps = instrument.observe(sky, write_outputs=False)[0]
freq_maps = freq_maps[:, 1:]  # Select polarization

## Define what you fit for
Create your sky model as a list of components

In [ ]:
components = [CMB(), Dust(150.), Synchrotron(20.)]

## Component separation
The tools inside ForeGroundBuster allow for very flexible and diverse component separation approaches. However, we also provide a set of predefined function that perform component separation out of the box. They suit most common use cases.

In [ ]:
result = basic_comp_sep(components, instrument, freq_maps)

## Explore the results
You have just solved for both the spectral parameters of your components and their amplitudes.

Get the spectral parameters name and values with

In [ ]:
print(result.params)
print(result.x)

You can also have a look at their semi-analytic covariance, but remember that it is accurate only the high signal-to-noise regime

In [ ]:
corner_norm(result.x, result.Sigma, labels=result.params)

The amplitudes of the components are stacked in the ``s`` attribute and they are in the same format of the input frequency maps: Q and U healpix maps, in this case.

In [ ]:
hp.mollview(result.s[0,1], title='CMB')
hp.mollview(result.s[1,1], title='Dust', norm='hist')
hp.mollview(result.s[2,1], title='Synchrotron', norm='hist')